# Java Frameworks

## Web Services

Un servicio web es un conjunto de protocolos abiertos y estandares para pasar datos entre aplicaciones

<ul>
<li>Funcion que esta disponible y es accesible por red(local o www)</li>
<li>Utilizan protocolos TCP/IP, HTTP, HTTPS</li>
<li>Los datos se transmiten en formato JSON o XML por lo que son independientes de cualquier lenguaje.</li>
</ul>

### Web Services structure

![ws-structure.PNG](ws-structure.PNG)


Un servicio web recibe datos por medio del request body y regresa datos por medio del response body.
![reqres.PNG](reqres.PNG)

![appsws.PNG](appsws.PNG)
    

### Data formats

#### Java Class (Spring boot)
```java
    public class CMUser {

	@Id
	@GeneratedValue(strategy = GenerationType.IDENTITY)
	@Column(name="Id")
	private Integer id;

	@Column(name="Name")
	@NotBlank(message = "Name is mandatory")
	private String name;

	@Column(name="Email", unique = true)
	@NotBlank(message = "Email is mandatory")
	private String email;
 
	@Column(name="Password")
	@NotBlank(message = "password is mandatory")
	private String password;
    
    //Getters & Setters

}
```

#### JSON(JavaScript Object Notation)
```javascript
    {
	"id": 1,
	"name": "Orlando Loza",
	"email": "orlandolozaarr@gmail.com",
	"password": "password"
    }
```

#### XML
```xml
    <userRequest> 
        <id>1</id>
        <name>Orlando Loza</name>
        <email>orlandolozaarr@gmail.com</email>
        <password>password</password>
    </userRequest>
```

## SOA(Service Oriented Application) Architecture vs Monolithic Architecture
![monolithic.PNG](monolithic.PNG)  ![soa.PNG](soa.PNG)






# Rest Services

![rest.PNG](rest.PNG)


## crear un RESTful web service en spring boot 

### Pasos para crear un un proyecto rest que guarde informacion de canciones y albums

<ul>
    <li>MySql tables</li>
    <li>Entity java classes</li>
    <li>Repository java classes</li>
    <li>Service logic java classes</li>
    <li>Rest Controller Java Classes</li>
    <li>Postman rest client test</li>
</ul>

### MySql tables
Tabla albums
```sql
CREATE TABLE `albums` (
  `Id` int(11) NOT NULL AUTO_INCREMENT,
  `Title` varchar(50) NOT NULL,
  `Release_Date` Date NOT NULL,
  `Price` varchar(100) NOT NULL,
  `Genre` varchar(150) NOT NULL,
  PRIMARY KEY (`Id`)
) ENGINE=InnoDB DEFAULT CHARSET=latin1;
```

Tabla songs
```sql
CREATE TABLE `songs` (
  `Id` int(11) NOT NULL AUTO_INCREMENT,
  `Album_Id` int(11) NOT NULL,
  `Title` varchar(50) NOT NULL,
  `Time` varchar(50) NOT NULL,
  PRIMARY KEY (`Id`),
  CONSTRAINT `sings_ibfk_1` FOREIGN KEY (`Album_Id`) REFERENCES `albums` (`Id`)
) ENGINE=InnoDB DEFAULT CHARSET=latin1;
```

### Entity
Las clases Entity representan en clases en java las tablas que existen en la base de datos.
por cada tabla existente en la base de datos necesitamos una clase Entity que este mapeada a esa tabla 
en donde los campos de la tabla los mapeamos a variables en la clase.

#### JPA & MySql Dependacies
Para poder crear una conexion a base de datos necesitamos librerias tanto de my sql como de hibernate que es el 
framework para base de datos que estaremos utilizando.

```xml
    <!-- JPA(Java Persistance API) library-->
    <dependency>
		<groupId>org.springframework.boot</groupId>
		<artifactId>spring-boot-starter-data-jpa</artifactId>
	</dependency>
    <!-- mysql library-->
    <dependency>
		<groupId>mysql</groupId>
		<artifactId>mysql-connector-java</artifactId>
		<scope>runtime</scope>
    </dependency>
```

Clases en java necesarias
Clase Album
```java
@Entity(name = "albums")
@JsonIgnoreProperties({"hibernateLazyInitializer", "handler"})
public class Album {

	@Id
	@GeneratedValue(strategy = GenerationType.IDENTITY)
	@Column(name="Id")
	private Integer id;

	@Column(name="Title")
	private String title;

	@Column(name="Release_Date", unique = true)
	private Date releaseDate;

	@Column(name="Price", unique = true)
	private String price;
	
	@Column(name="Genre", unique = true)
	private String genre;
    
    //Setters & Getters
}
```

Clase Song
```java
@Entity(name = "songs")
@JsonIgnoreProperties({"hibernateLazyInitializer", "handler"})
public class Song {

	@Id
	@GeneratedValue(strategy = GenerationType.IDENTITY)
	@Column(name="Id")
	private Integer id;
    
    @Column(name="Album_Id")
	private Integer albumId;

	@Column(name="Title")
	private String title;

	@Column(name="Time")
	private String time;

	@Column(name="Genre")
	private String genre;
    
    //Setters & Getters
}
```

### Repository

los repositorios en spring boot son las conexiones a la base de datos utilizando Hibernate y JPA.
son interfaces en java que heredan de JPARepository<Entity, Primary Key>
al heredar de esta clase y apuntar a la entidad que queremos utilizar automaticamente tenemos las operaciones 
basicas tipo CRUD(Create, Read, Update, Delete).

```java
public interface AlbumRepository extends JpaRepository<Album, Integer> {
	
}
```

```java
public interface SongRepository extends JpaRepository<Song, Integer> {
	
}
```

### Service

En las clases service escribimos la logica de nuestros servicios, es en estas clases en las que utilizamos los metodos
CRUD del repositorio para comunicarnos con la base de datos.

```java
@Service
public class AlbumService {
	
	@Autowired
	private AlbumRepository albumRepository;
	
	public void createAlbum(Album album) {
		albumRepository.save(album);
	}
	
	public List<Album> findAll(){
		return albumRepository.findAll();
	}
    
    //metodo para encontrar un album por su id
    
    //metodo para borrar registros por id
    
    //metodo para actualizar registros por su id
	
}
```



### Controller

Las clases controller son las que exponen lus servicios a internet, le añaden un endpoint al las funciones que tenemos
en las clases service. estas clases no tienen logica simplemente ejecutan los metodos que tenemos en el service.

```java
@RestController
@RequestMapping("/api/v1/album") //endpoint para llamar este web service desde internet ya sea de un navegador o desde postman
public class AlbumController {
	
	@Autowired
	private AlbumService albumService; 
	
	@PostMapping
	public void createAlbum(@RequestBody Album album) {
		albumService.createAlbum(album);
	}
	
	@GetMapping
	public List<Album> findAll(){
		return albumService.findAll();
	}
	
}
```

### application.properties

Para poder conectarnos a la base de datos que teneos en aws es necesario agregar las credenciales y drivers de mysql 
en las propiedades del proyecto.
en un proyecto Spring estas propiedades se encuentran en la carpeta src.main.resources
![app-properties.PNG](app-properties.PNG)

abrimos el archivo application.properties y agregamos lo siguiente
```properties
# Mysql credentials
spring.jpa.database-platform=org.hibernate.dialect.MySQL5Dialect
spring.jpa.hibernate.ddl-auto=none
spring.jpa.show-sql=true
spring.datasource.url= jdbc:mysql://javaframeworks-db.cv0ojbio2vk5.us-east-1.rds.amazonaws.com:3306/tecgurus?useSSL=false

# Username & password
spring.datasource.username=admin
spring.datasource.password=password

```


### Testing in Postman

Postman es un cliente para servicios rest en donde podemos probar y mandar peticiones a los servicios web que estamos creando.

![postman-client.png](postman-client.png)

<ol>
    <li>URL del web service</li>
    <li>tipo de request (POST, GET, PUT, DELETE)</li>
    <li>boton Send para mandar llamar el web service</li>
    <li>respuesta del api</li>
    <li>información de tiempo de respuesta y status code de la respuesta</li>
    <li>Historial requests</li>
</ol>


tenemos dos web services que podemos probar createAlbum y findAll y su endpoint es /api/v1/album

#### Metodo POST

```java
    @PostMapping
	public void createAlbum(@RequestBody Album album) {
		albumService.createAlbum(album);
	}
	
	@GetMapping
	public List<Album> findAll(){
		return albumService.findAll();
	}
```

El metodo POST createAlbum nos pide de parametro un objeto Album por lo que hay que agregarlo en el request en formato json
```javascript
     {
        "title": "album title",
        "releaseDate": "2019",
        "price": "300",
        "genre": "pop"
     }
```

en postman justo debajo del url esta el campo para el request body que es donde pondremos el json, seleccionamos la pestaña
body y escibimos ahi nuestro objeto en formato json y presionamos el boton send para mandar el request a java.
![postman2.PNG](postman2.PNG)

podemos cambiar el valor de los datos que estamos enviando y volver a mandar la peticion para crear mas registros en la base de datos.


#### Metodo GET

Para mandar llamar el metodo GET findAll lo unico que tenemos que hacer es cambiar ahora el tipo de metodo a GET 
y con el mismo URL volvemos a dar click en el boton Send.
En el response nos debe aparecer un json con los registros que encontro en la base de datos.

![postman3.PNG](postman3.PNG)


Podemos comprobar que en la base de datos en mysql existen los registros que creamos desde postman y que son los mismos que nos regresa el metodo GET.

![sql.PNG](sql.PNG)